In [6]:
import os
import argparse
from time import time
import pandas as pd
from sqlalchemy import create_engine

In [8]:
# Parameters
user = 'root'
password = 'root'
host = 'localhost' 
port = '5432' 
db = 'ny_taxi'
table_name = 'taxi_zone_lookup'
url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-09.csv.gz' 
csv_name = 'zone.csv'

In [ ]:
# Download CSV
if url.endswith('.csv.gz'):
    csv_name = 'output.csv.gz'
else:
    csv_name = 'output.csv'

os.system(f"wget {url} -O {csv_name}")

In [10]:
# DB connection
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

df_iter = pd.read_csv(csv_name, iterator=True, chunksize=100000, low_memory=False)

df = next(df_iter)

# df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
# df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

df.head(n=0).to_sql(name=table_name, con=engine, if_exists='replace')

df.to_sql(name=table_name, con=engine, if_exists='append')

while True: 

        try:
            t_start = time()
            
            df = next(df_iter)

            df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
            df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

            df.to_sql(name=table_name, con=engine, if_exists='append')

            t_end = time()

            print('inserted another chunk, took %.3f second' % (t_end - t_start))

        except StopIteration:
            print("Finished data ingestion.")
            break

Finished data ingestion.
